## Yelp Challenge

Dataset Documentation: <br>
https://www.yelp.com/dataset/documentation/main

In [1]:
import pandas as pd
import tarfile
from tqdm import tqdm
import json

In [2]:
zf = tarfile.open('yelp_dataset.tar') 
#df = pd.read_csv(zf.open('intfile.csv'))
for name in zf.list():
    print (name)

?rw-r--r-- daniellg/users  138279749 2018-11-15 11:22:39 business.json 
?rw-r--r-- daniellg/users  408807658 2018-11-15 11:25:00 checkin.json 
?rw-r--r-- daniellg/users 5347475638 2018-11-15 11:35:37 review.json 
?rw-r--r-- daniellg/users  244535478 2018-11-15 11:26:18 tip.json 
?rw-r--r-- daniellg/users 2485747393 2018-11-15 11:24:48 user.json 
?rw-r--r-- daniellg/users   25661152 2019-01-11 19:06:09 photo.json 
?rw-r--r-- daniellg/users     101186 2019-01-14 11:31:35 Dataset_Challenge_Dataset_Agreement.pdf 
?rw-r--r-- daniellg/users     111822 2019-01-14 11:35:09 Yelp_Dataset_Challenge_Round_13.pdf 


TypeError: 'NoneType' object is not iterable

In [3]:
## Feel free to extract more files here
#zf.extract("review.json")

In [2]:
line_count = len(open("review.json").readlines())
user_ids, business_ids, stars, dates, texts = [], [], [], [], []
with open("review.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        user_ids += [blob["user_id"]]
        business_ids += [blob["business_id"]]
        stars += [blob["stars"]]
        dates += [blob["date"]]
        texts += [blob["text"]]
ratings = pd.DataFrame(
    {"user_id": user_ids, "business_id": business_ids, "rating": stars, "date": dates, "text": texts}
)
user_counts = ratings["user_id"].value_counts()
active_users = user_counts.loc[user_counts >= 5].index.tolist()

100%|██████████| 6685900/6685900 [01:08<00:00, 97704.32it/s] 


In [3]:
active_users[0:5]

['CxDOIDnH8gp9KXzpBHJYXw',
 'bLbSNkLggFnqwNNzzq-Ijw',
 'PKEzKWv_FktMm2mGPjwd0Q',
 'ELcQDlf69kb-ihJfxZyL0A',
 'DK57YibC5ShBmqQl97CKog']

In [4]:
ratings.head()

,user_id,business_id,rating,date,text
0,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,2013-05-07 04:34:36,Total bill for this horrible service? Over $8G...
1,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,2017-01-14 21:30:33,I *adore* Travis at the Hard Rock's new Kelly ...
2,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,2016-11-09 20:09:03,I have to say that this office really has it t...
3,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,2018-01-09 20:56:38,Went in for a lunch. Steak sandwich was delici...
4,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,2018-01-30 23:07:38,Today was my second out of three sessions I ha...


In [5]:
ratings.shape

(6685900, 5)

In [6]:
n_users = len(ratings.user_id.unique())
n_restaurants = len(ratings.business_id.unique())
print('Unique Users: {0}, unique restaurants: {1}'.format(n_users, n_restaurants))

Unique Users: 1637138, unique restaurants: 192606


### Notes

1. Baseline:
User based (Bowen)
ALS (later)


1. Cold start: (<5 reviews)
content based (Nearest neighbors on review text, metadata tex) (Zhongling)


2. Main model
Field-aware factorization machine (James)
Locality Sensitive Hashing (Ujjwal)
Collective Matrix Factorization (Bowen)


Hybrid approach
3. Metrics
TBD

### Train / Holdout 

The rating dataset has ~6.6 million rows and is time consuming to perform group by & aggregation operations, which is required for constructing holdout set. Therefore, I randomly subsample 1/10 rows in order to make code run through more quickly. Note that this is not an optimal practice, I did it solely because I want to speed up the data pre-processing and modeling cycle. 

In [20]:
# ratings_sample = ratings.sample(frac= 1/10, replace=False, random_state=1)
# Downsample by users
user_id_unique = ratings.user_id.unique()
user_id_sample = pd.DataFrame(user_id_unique, columns=['unique_user_id']) \
                    .sample(frac= 1/10, replace=False, random_state=1)

In [24]:
ratings_sample = ratings.merge(user_id_sample, left_on='user_id', right_on='unique_user_id') \
                    .drop(['unique_user_id'], axis=1)
print(ratings_sample.head())
print(ratings_sample.shape)

                  user_id             business_id  rating  \
0  w31MKYsNFMrjhWxxAb5wIw  eU_713ec6fTGNO4BegRaww     4.0   
1  sG_h0dIzTKWa3Q6fmb4u-g  b2jN2mm9Wf3RcrZCgfo1cg     2.0   
2  sG_h0dIzTKWa3Q6fmb4u-g  3FG0ZCPkHzDxP8UOmoEyiQ     1.0   
3  sG_h0dIzTKWa3Q6fmb4u-g  4IBXNxLsEcfUIx6LUM2Aqw     2.0   
4  sG_h0dIzTKWa3Q6fmb4u-g  oq9bF1KUzSldObMSQ_aalQ     5.0   

                  date                                               text  
0  2013-01-20 13:25:59  I'll be the first to admit that I was not exci...  
1  2015-01-18 14:04:18  I was really looking forward to visiting after...  
2  2016-07-09 22:57:25  Went there to buy a car today. Walked around t...  
3  2016-01-10 19:25:08  55 min wait for takeout and a $7 draft beer se...  
4  2017-01-20 18:31:41  Best Philly Cheese Steak outside of Philly. Be...  
(658169, 5)


In [25]:
#ratings_sample.to_csv('ratings_sample.csv')

In [8]:
# hold out last review
ratings_user_date = ratings_sample.loc[:, ['user_id', 'date']]
index_holdout = ratings_user_date.groupby(['user_id'], sort=False)['date'].transform(max) == ratings_user_date['date']
ratings_holdout = ratings_sample[index_holdout]
ratings_train = ratings_sample[~index_holdout]

In [9]:
print('There are {0} rows, {1} columns in training set.'.format(ratings_train.shape[0], ratings_train.shape[1]))
print('There are {0} rows, {1} columns in holdout set.'.format(ratings_holdout.shape[0], ratings_holdout.shape[1]))

There are 279080 rows, 5 columns in training set.
There are 389510 rows, 5 columns in holdout set.
